### **AML - Homework 4 Question 2**

In [ ]:
# INSTALL THESE LIBRARIES IF NOT ALREADT INSTALLED

# %%bash
# pip install nltk
# pip install datasets
# pip install transformers[torch]
# pip install tokenizers
# pip install evaluate
# pip install rouge_score
# pip install sentencepiece
# pip install huggingface_hub
# pip install transformers datasets wandb accelerate

In [ ]:
# !pip install tf-keras

In [ ]:
# !pip install transformers datasets wandb accelerate

In [2]:
# !pip install datasets

In [2]:
import wandb
from datasets import Dataset
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainer, Seq2SeqTrainingArguments,DataCollatorForSeq2Seq


### **Q1.**

Pick an existing fine-tuning dataset (or create your own - you can create it by scraping the web or using some of the documents you have on your computer (do not use anything confidential or copyrighted) or by asking ChatGPT to generate the data). Mention which dataset you picked/created and briefly describe the dataset. [10 points]

### **Answer**:

For this task, our group chose the **"yahoo_answers_qa"** dataset from the datasets library in Python.

This dataset is a comprehensive collection of question and answer pairs sourced from the Yahoo! Answers platform, covering diverse categories such as Society & Culture, Health, Science, and Technology. Each entry includes the question title, question content, best answer, and category.

The datasets library, developed by Hugging Face, provides a simple and efficient way to load and process large datasets, making it a valuable tool for natural language processing tasks.

Due to memory concerns, we used only 2% of the dataset, which still provides a rich and diverse set of examples. After selecting this subset, we split it into training and validation sets to use in the fine-tuning process. We did this to manage computational resources and avoid the memory run out error.

The data is loaded in Q2.

### **Link to our Wandb project report**

https://api.wandb.ai/links/nsarwar-indiana-university/rwuur7jx


### **Q2.**

Fine-tune an open-source pre-trained conversational language model of your choice (that you can take, e.g., from the Hugging Face Transformers library) with the dataset you picked or created. Make sure the model you pick has at least 500M parameters. [20 points]  Connect to wandb and to track the progress of your fine-tuning (e.g. your training loss). Share the link to your wandb project with us in the report you submit (see here for how to do it: https://wandb.ai/ivangoncharov/wandb-teams-for-students/reports/How-to-Use-W-B-Teams-For-Your-University-Machine-Learning-Projects-For-Free---VmlldzoxMjk1MjkxLinks to an external site.) [10 points] Test your model on a few prompts before and after fine-tunining and report any interesting differences. If you didn't observe any interesting differences, comment on why not. [10 points]

Setting up wandb (it will ask for API to connect with wandb dashboard)

In [3]:
# Step 2: Set up WandB for Experiment Tracking
wandb.login()  # Login to WandB (you'll need an API key from your WandB account)
wandb.init(project="flan-t5-fine-tuning", name="flan-t5-qa")

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: nsarwar (nsarwar-indiana-university). Use `wandb login --relogin` to force relogin


We chose the FLAN-T5 Model for this HW. Loading the FLAN-T5 Model:


In [4]:
# Step 3: Load the Pre-Trained Model and Tokenizer
MODEL_NAME = "google/flan-t5-large"  # Using Flan-T5 Large model (500M+ parameters)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Testing the model on different input prompts- Before Fine tunning

In [5]:
test_inputs = [
    "Given the context of a 12-year-old attending middle school, answer the question: How old are you when you start high school?",
    "Answer the question: How can I solve my Python code problem?",
    "Answer the question: How can I print a hello world in java?",
    "Based on common advice, how can someone effectively prepare for a job interview in the tech industry?",
    "How can I prepare for a tech job interview?",
    "How to read a CSV file in Python using pandas?",
    "How to create a simple RESTful API in Python with Flask?"
]

print("Testing the model before fine-tuning")
for input_text in test_inputs:
    input_ids = tokenizer(input_text, return_tensors="pt", truncation=True).input_ids
    outputs = model.generate(input_ids, max_length=50)
    print(f"Input: {input_text}")
    print(f"Output: {tokenizer.decode(outputs[0], skip_special_tokens=True)}")

Testing the model before fine-tuning
Input: Given the context of a 12-year-old attending middle school, answer the question: How old are you when you start high school?
Output: 15
Input: Answer the question: How can I solve my Python code problem?
Output: Use a recursive method.
Input: Answer the question: How can I print a hello world in java?
Output: Use a java.split() function to print the hello world.
Input: Based on common advice, how can someone effectively prepare for a job interview in the tech industry?
Output: Prepare for the interview by reading the job description and preparing for the interview.
Input: How can I prepare for a tech job interview?
Output: You can use a resume and cover letter to impress the interviewer.
Input: How to read a CSV file in Python using pandas?
Output: csv = [int(x) for x in input().split()] csv.append(csv)
Input: How to create a simple RESTful API in Python with Flask?
Output: Using Flask, you can create a simple RESTful API in Python.


Loading the dataset "yahoo_answers_qa" to fine tune the model:

In [7]:

DATA_NAME = "yahoo_answers_qa"
stackoverflow_python = load_dataset(DATA_NAME, trust_remote_code=True)

# The dataset is very large, I am reducing the size of training data

shuffled_dataset = stackoverflow_python['train'].shuffle(seed=42)

# Selecting 2% of the dataset
num_samples = int(0.02 * len(shuffled_dataset))
reduced_dataset = shuffled_dataset.select(range(num_samples))

# Replacing the train dataset with the reduced dataset
stackoverflow_python['train'] = reduced_dataset

# dividing the dataset to train and test data
stackoverflow_python = stackoverflow_python["train"].train_test_split(test_size=0.3)

stackoverflow_python

In [9]:
def preprocess_function(examples):
    inputs = examples["question"]
    targets = examples["answer"]
    model_inputs = tokenizer(inputs, text_target=targets, max_length=512, truncation=True)
    return model_inputs

 tokenization of the dataset:



In [10]:
tokenized_dataset = stackoverflow_python.map(preprocess_function, batched=True)

Map:   0%|          | 0/1222 [00:00<?, ? examples/s]

Map:   0%|          | 0/525 [00:00<?, ? examples/s]

preparing data collator:

In [11]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

**Fine- tunning the model:**

In [12]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./flan-t5-finetuned",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    num_train_epochs=10,
    weight_decay=0.01,
    save_total_limit=2,
    predict_with_generate=True,
    logging_dir="./logs",
    logging_steps=100,
    save_steps=500,
    # fp16=True,
    bf16=True,
    report_to="wandb"
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


**Defining the Trainer:**

In [13]:
trainer = Seq2SeqTrainer(
    model=model,                         # Pre-trained model
    args=training_args,                  # Training arguments
    train_dataset=tokenized_dataset["train"],     # Tokenized training dataset
    eval_dataset=tokenized_dataset["test"],      # Tokenized evaluation dataset
    tokenizer=tokenizer,                 # Tokenizer
    data_collator=data_collator          # Handles padding dynamically
)


<ipython-input-13-a38181dec01a>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Ensuring that both the model and inputs are stored on gpu (same device):

In [14]:
import torch

# Explicitly set the device to CPU
device = torch.device("cuda")

# Move the model to CPU
model.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 1024)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 1024)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=1024, out_features=1024, bias=False)
              (k): Linear(in_features=1024, out_features=1024, bias=False)
              (v): Linear(in_features=1024, out_features=1024, bias=False)
              (o): Linear(in_features=1024, out_features=1024, bias=False)
              (relative_attention_bias): Embedding(32, 16)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=1024, out_features=2816, bias=False)
              (wi_1): Linear(in_features=1024, out_features=2816, bias=False)
       

**Training the model on yahoo dataset (with the specified parameters in training_args earlier) :**

In [15]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
0,3.212700,3.039998
1,2.992600,3.035279
2,2.862400,3.039718
3,2.739900,3.051322
4,2.613100,3.065259
5,2.544600,3.085451
6,2.501900,3.096652
7,2.419800,3.109341
8,2.406900,3.117339
9,2.370500,3.120727


TrainOutput(global_step=1520, training_loss=2.665618361924824, metrics={'train_runtime': 1977.0819, 'train_samples_per_second': 6.181, 'train_steps_per_second': 0.769, 'total_flos': 978202938138624.0, 'train_loss': 2.665618361924824, 'epoch': 9.965630114566284})

The training loss is decreasing. However the validation loss keeps increasing a little. The reason could be a very small training dataset. Due to memory issues we have kept the training data to minimum size.

In [16]:
#Save the Fine-Tuned Model
model.save_pretrained("./flan-t5-finetuned")
tokenizer.save_pretrained("./flan-t5-finetuned")

('./flan-t5-finetuned/tokenizer_config.json',
 './flan-t5-finetuned/special_tokens_map.json',
 './flan-t5-finetuned/spiece.model',
 './flan-t5-finetuned/added_tokens.json',
 './flan-t5-finetuned/tokenizer.json')

**Testing the Model with same input prompts After Fine-Tuning**

In [17]:
print("\nTesting the  model after fine-tuning")
fine_tuned_model = AutoModelForSeq2SeqLM.from_pretrained("./flan-t5-finetuned")
fine_tuned_tokenizer = AutoTokenizer.from_pretrained("./flan-t5-finetuned")

for input_text in test_inputs:
    input_ids = fine_tuned_tokenizer(input_text, return_tensors="pt", truncation=True).input_ids
    outputs = fine_tuned_model.generate(input_ids, max_length=50)
    print(f"Input: {input_text}")
    print(f"Output: {fine_tuned_tokenizer.decode(outputs[0], skip_special_tokens=True)}")


Testing the  model after fine-tuning
Input: Given the context of a 12-year-old attending middle school, answer the question: How old are you when you start high school?
Output: You will be 13 years old when you start high school.
Input: Answer the question: How can I solve my Python code problem?
Output: if you are using a python interpreter, you can use a python interpreter to compile your code. . . if you are using a python
Input: Answer the question: How can I print a hello world in java?
Output: if you are using java 1.6 or higher you can use javascript.split(" ") to print the hello world. . . if you are using java 1.8 or higher you can use
Input: Based on common advice, how can someone effectively prepare for a job interview in the tech industry?
Output: You'll need to know what you're talking about. You'll also need to know what you're talking about. You'll need to know what you're talking about.
Input: How can I prepare for a tech job interview?
Output: You can't prepare for a 

### **Thoughts and Discussion**

* We chose **google/flan-t5-large** pre trained model, which has 780 million parameters.

* **Link to wandb project:**

https://api.wandb.ai/links/nsarwar-indiana-university/rwuur7jx



* Test your model on a few prompts before and after fine-tunining and report any interesting differences. If you didn't observe any interesting differences, comment on why not. [10 points]

We have tested the model on different input prompts before and after tunning (shown above in code).

**Insights on the fine tuned model**
After fine-tuning, the model provided more detailed but often repetitive and less coherent responses. The increased wordiness and repetition suggest overfitting on the smaller dataset, indicating a need for further fine-tuning or data augmentation. This was expected because our training dataset was very small.  

Before fine-tuning, the model provided brief and sometimes inaccurate responses. After fine-tuning, the model's responses were more detailed but often contained repetition or incomplete sentences. This could indicate that the model started to overfit on the smaller dataset, learning patterns specific to it but losing some generalization capability. The increase in wordiness also suggest that the model tried to provide more context but struggled with coherency, pointing to the need for further fine-tuning or data augmentation to achieve better performance.


### END of Homework 4 Question 2

--------------